In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from glob import glob

from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel, preprocess_input

In [ ]:
train_path = 'data/train'
valid_path = 'data/test'

In [ ]:
IMAGE_SIZE = [200,200]

In [ ]:
image_files = glob(train_path + r'/*/*.jpg')
valid_image_files = glob(valid_path + r'/*/*.jpg')

In [ ]:
folders = glob(train_path + '/*')

In [ ]:
folders

In [ ]:
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

In [ ]:
ptm = PretrainedModel(input_shape=IMAGE_SIZE + [3],
                      weights='imagenet',
                      include_top=False)

In [ ]:
ptm.trainable = False

In [ ]:
K = len(folders)
x = Flatten()(ptm.output)
x = Dense(K, activation='softmax')(x)

In [ ]:
model = Model(inputs=ptm.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
gen = ImageDataGenerator(rotation_range=20,
                         width_shift_range=0.1,
                         height_shift_range=0.1, 
                         shear_range=0.1, 
                         zoom_range=0.2,
                         horizontal_flip=True,
                         preprocessing_function=preprocess_input)

In [ ]:
batch_size = 32
train_generator = gen.flow_from_directory(
    train_path,
    shuffle=True,
    target_size=IMAGE_SIZE,
    batch_size=batch_size
)
valid_generator = gen.flow_from_directory(
    valid_path,
    target_size=IMAGE_SIZE,
    batch_size=batch_size
)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
r = model.fit_generator(
    train_generator,
    validation_data=valid_generator,
    epochs=3,
    steps_per_epoch=int(np.ceil(len(image_files)/batch_size)),
    validation_steps=int(np.ceil(len(valid_image_files)/batch_size))
)